In [1]:
# ! pip install -e ../../savo

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
from copy import deepcopy as copy
from scipy import optimize
import torch

import sys
sys.path.append('../')
sys.path.append('../machineIO/')
from savo import savo
from machineIO.utils import plot_2D_projection, dictClass
from machineIO.objFunc import SingleTaskObjectiveFunction
from machineIO.VM import VirtualMachineIO
from machineIO import Evaluator

Failed to import 'epics'
Failed to import 'phantasy'


In [3]:
ndim = 50
budget = 200
ninit = 25  # number of initial ES steps to collect data for surrogate model training
noise = 0.0
max_dx = 0.01
lr = 1e-2

In [4]:
input_CSETs = [f'X{i}:I_CSET' for i in range(ndim)]
input_RDs   = [f'X{i}:I_RD' for i in range(ndim)]
input_min = -np.ones(ndim)
input_max =  np.ones(ndim)
input_tols = 1e-3*(input_max-input_min)
output_RDs = [f'Y{i}:I_RD' for i in range(2)]
output_min = -2*np.ones(2)
output_max =  2*np.ones(2)

In [5]:
vmio = VirtualMachineIO(
    control_CSETs = control_CSETs,
    input_RDs = input_RDs,
    monitor_PVs = output_RDs,
    input_min = input_min,
    input_max = input_max,
    monitor_min = output_min,
    monitor_max = output_max,
)

TypeError: VirtualMachineIO.__init__() got an unexpected keyword argument 'input_CSETs'

# obj_func

: rastirigin over 2D latent space of random NN for both high-dim and visualization

In [ ]:
def rastrigin(x,noise=noise):
    x = torch.as_tensor(x)
    b,d = x.shape
    y = torch.sum(x**2 - torch.cos(2*np.pi*x),axis=1)/d +1
    return 1-y + torch.randn(b)*noise

In [ ]:
grid = np.linspace(-2,2,128)
x1,x2 = np.meshgrid(grid,grid)
xgrid = np.vstack((x1.flatten(), x2.flatten())).T
ygrid = rastrigin(xgrid,noise=0)
def plot_contour():
    fig,ax = plt.subplots(figsize=(4,3.3),dpi=128)
    cs = ax.tricontourf(xgrid[:,0],xgrid[:,1],ygrid,levels=32)
    fig.colorbar(cs,ax=ax)
    return fig,ax

In [ ]:
objfunc = SingleTaskObjectiveFunction(
    objective_PVs = output_RDs,
    composite_objective_name = 'rastrigin',
    custom_function = rastrigin,
    objective_goal = None, 
    objective_weight = None,
    objective_tolerance = None,
)

In [ ]:
evaluator = Evaluator(
    machineIO = vmio,
    input_CSETs = input_CSETs,
    input_RDs = input_RDs,
    input_tols = input_tols,
    output_RDs = output_RDs,
    df_manipulators = [objfunc.calculate_objectives_from_df],
)

# SAVO

In [ ]:
x0 = torch.rand(input_max.shape)*(input_max-input_min) + input_min

In [ ]:
sv = savo(x0, max_dx, input_min, input_max, evaluator)

In [ ]:
sv.runES(200)
fig, ax = plt.subplots(figsize=(4,3),dpi=128)
ax.plot(sv.history['y']);
ax.set_xlabel('epoch');
ax.set_ylabel('objective');

fig, ax = plot_contour()
ax.plot(sv['x_history'][i][:,0],B['x_history'][i][:,1],color='C'+str(i),lw=0.2,alpha=0.3)

In [ ]:
fig, ax = plt.subplots(figsize=(4,3),dpi=128)
ax.plot(obj_func.history['y']);
ax.set_xlabel('epoch');
ax.set_ylabel('objective');

In [ ]:
fig, ax = plt.subplots(figsize=(3.3*1.5,2.5*1.5),dpi=256)
cs = ax.tricontourf(x[:,0],x[:,1],y, levels=64, cmap="viridis");
fig.colorbar(cs,ax=ax,shrink=0.95)
z = np.array(obj_func.history['z'])
ax.scatter(z[:,0],z[:,1],color='k',s=2)
for i in range(len(z) - 1):
    dx = z[i + 1, 0] - z[i, 0]
    dy = z[i + 1, 1] - z[i, 1]
    distance = np.sqrt(dx**2 + dy**2)
    # Scale arrowhead size based on the distance
    head_size = distance * 0.28  # Adjust 0.2 as a scaling factor if needed
    ax.arrow(z[i, 0]+0.2*dx, z[i, 1]+0.2*dy, 
              0.6*dx, 0.6*dy,
              color='r', alpha=0.7, 
              width = 0.02*head_size,
              head_width=0.8*head_size, 
              head_length=head_size, 
              length_includes_head=True)
ax.scatter(z[0,0],z[0,1],marker='x',color='orange')
ax.set_xlim(z[:,0].min()-0.1,z[:,0].max()+0.1);
ax.set_ylim(z[:,1].min()-0.1,z[:,1].max()+0.1);
fig.tight_layout()

In [ ]:
obj_SAVO = obj_func

# SAVO with Adam

In [ ]:
obj_func = const_obj_func()
sv = savo(obj_func= obj_func,
          x0 = x0,
          max_dx = max_dx,
          x_bounds = x_bounds,
          )

In [ ]:
sv.runES(ninit)
sv.run_savo(budget-ninit,lambdaES=1,lr=lr,adam=True)

In [ ]:
fig, ax = plt.subplots(figsize=(4,3),dpi=128)
ax.plot(obj_func.history['y']);
ax.set_xlabel('epoch');
ax.set_ylabel('objective');

In [ ]:
fig, ax = plt.subplots(figsize=(3.3*1.5,2.5*1.5),dpi=256)
cs = ax.tricontourf(x[:,0],x[:,1],y, levels=64, cmap="viridis");
fig.colorbar(cs,ax=ax,shrink=0.95)
z = np.array(obj_func.history['z'])
ax.scatter(z[:,0],z[:,1],color='k',s=2)
for i in range(len(z) - 1):
    dx = z[i + 1, 0] - z[i, 0]
    dy = z[i + 1, 1] - z[i, 1]
    distance = np.sqrt(dx**2 + dy**2)
    # Scale arrowhead size based on the distance
    head_size = distance * 0.28  # Adjust 0.2 as a scaling factor if needed
    ax.arrow(z[i, 0]+0.2*dx, z[i, 1]+0.2*dy, 
              0.6*dx, 0.6*dy,
              color='r', alpha=0.7, 
              width = 0.02*head_size,
              head_width=0.8*head_size, 
              head_length=head_size, 
              length_includes_head=True)
ax.scatter(z[0,0],z[0,1],marker='x',color='orange')
ax.set_xlim(z[:,0].min()-0.1,z[:,0].max()+0.1);
ax.set_ylim(z[:,1].min()-0.1,z[:,1].max()+0.1);
fig.tight_layout()

In [ ]:
obj_ADAM = obj_func

# SAVO without ES

In [ ]:
obj_func = const_obj_func()
sv = savo(obj_func= obj_func,
          x0 = x0,
          max_dx = max_dx,
          x_bounds = x_bounds,
          )

In [ ]:
sv.runES(ninit)
sv.run_savo(budget-ninit,lambdaES=0,lr=lr)

In [ ]:
fig, ax = plt.subplots(figsize=(4,3),dpi=128)
ax.plot(obj_func.history['y']);
ax.set_xlabel('epoch');
ax.set_ylabel('objective');

In [ ]:
fig, ax = plt.subplots(figsize=(3.3*1.5,2.5*1.5),dpi=256)
cs = ax.tricontourf(x[:,0],x[:,1],y, levels=64, cmap="viridis");
fig.colorbar(cs,ax=ax,shrink=0.95)
z = np.array(obj_func.history['z'])
ax.scatter(z[:,0],z[:,1],color='k',s=2)
for i in range(len(z) - 1):
    dx = z[i + 1, 0] - z[i, 0]
    dy = z[i + 1, 1] - z[i, 1]
    distance = np.sqrt(dx**2 + dy**2)
    # Scale arrowhead size based on the distance
    head_size = distance * 0.28  # Adjust 0.2 as a scaling factor if needed
    ax.arrow(z[i, 0]+0.2*dx, z[i, 1]+0.2*dy, 
              0.6*dx, 0.6*dy,
              color='r', alpha=0.7, 
              width = 0.02*head_size,
              head_width=0.8*head_size, 
              head_length=head_size, 
              length_includes_head=True)
ax.scatter(z[0,0],z[0,1],marker='x',color='orange')
ax.set_xlim(z[:,0].min()-0.1,z[:,0].max()+0.1);
ax.set_ylim(z[:,1].min()-0.1,z[:,1].max()+0.1);
fig.tight_layout()

In [ ]:
obj_SAVO_noES = obj_func

# SAVO ADAM without ES

In [ ]:
obj_func = const_obj_func()
sv = savo(obj_func= obj_func,
          x0 = x0,
          max_dx = max_dx,
          x_bounds = x_bounds,
          )

In [ ]:
sv.runES(ninit)
sv.run_savo(budget-ninit,lambdaES=0,lr=lr,adam=True)

In [ ]:
fig, ax = plt.subplots(figsize=(4,3),dpi=128)
ax.plot(obj_func.history['y']);
ax.set_xlabel('epoch');
ax.set_ylabel('objective');

In [ ]:
fig, ax = plt.subplots(figsize=(3.3*1.5,2.5*1.5),dpi=256)
cs = ax.tricontourf(x[:,0],x[:,1],y, levels=64, cmap="viridis");
fig.colorbar(cs,ax=ax,shrink=0.95)
z = np.array(obj_func.history['z'])
ax.scatter(z[:,0],z[:,1],color='k',s=2)
for i in range(len(z) - 1):
    dx = z[i + 1, 0] - z[i, 0]
    dy = z[i + 1, 1] - z[i, 1]
    distance = np.sqrt(dx**2 + dy**2)
    # Scale arrowhead size based on the distance
    head_size = distance * 0.28  # Adjust 0.2 as a scaling factor if needed
    ax.arrow(z[i, 0]+0.2*dx, z[i, 1]+0.2*dy, 
              0.6*dx, 0.6*dy,
              color='r', alpha=0.7, 
              width = 0.02*head_size,
              head_width=0.8*head_size, 
              head_length=head_size, 
              length_includes_head=True)
ax.scatter(z[0,0],z[0,1],marker='x',color='orange')
ax.set_xlim(z[:,0].min()-0.1,z[:,0].max()+0.1);
ax.set_ylim(z[:,1].min()-0.1,z[:,1].max()+0.1);
fig.tight_layout()

In [ ]:
obj_ADAM_noES = obj_func

# plot all

In [ ]:
obj_funcs = [obj_NM, obj_ES, obj_SAVO, obj_ADAM, obj_SAVO_noES, obj_ADAM_noES]
obj_funcs_labels = ['Nelder-Mead', 'ES', 'ES+SG', 'ES+adamSG', 'SG', 'adamSG']

In [ ]:
fig, ax = plt.subplots(figsize=(3.3*1.5,1),dpi=96)
i = 0
for obj_func, label in zip(obj_funcs,obj_funcs_labels) :
    if  i==1:
        break
    ax.plot(obj_func.history['y'],label=label);
    i+=1
ax.plot(obj_func.history['y'],label='others');
ax.set_xlabel('epoch');
ax.set_ylabel('objective');
ax.set_ylim(-1.26,-1.22)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
fig, ax = plt.subplots(figsize=(3.3*1.5,2*1.5),dpi=96)
for obj_func, label in zip(obj_funcs,obj_funcs_labels) :
    ax.plot(obj_func.history['y'],label=label);
ax.set_xlabel('epoch');
ax.set_ylabel('objective');
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))